In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import numpy as np
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import cv2
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

use_cuda = torch.cuda.is_available()
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

0
1
GeForce RTX 3080


# Dataset Definitions

In [2]:
"""
Mean and standard deviation of pixel values to be used for image normalization.
"""
mean = [0.2773, 0.2589, 0.2684]
std = [0.1650, 0.1639, 0.1658]



"""
Transformations for training datasets are randomly augmented with 
horizontal flips, rotations and brightness adjustments.

Images in test datasets are preserved as-is.
"""
train_transform = transforms.Compose([transforms.ToPILImage(),
                                      transforms.Resize((224, 224)),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.RandomRotation(degrees=30),
                                      transforms.ColorJitter(brightness=0.2),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean,std)
                                      ])



test_transform = transforms.Compose([transforms.ToPILImage(),
                                     transforms.Resize((224, 224)),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean,std)
                                    ])

In [3]:
class ChimpNamesDataset(Dataset):
    IMG_SIZE = 224
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.name_labels = self.annotations['name'].values
        
        #Encoding name strings into unique integers to be used as labels.
        self.name_set = sorted(set(self.name_labels))
        self.name_dict = {}
        for i in range(len(self.name_set)):
            self.name_dict[self.name_set[i]] = i
            
        self.annotations['labels'] = self.annotations.name.map(self.name_dict)
        self.annotations.head()

        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, index):
        
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = cv2.imread(img_path)
        name_label = self.annotations.iloc[index, 10]
        
        if self.transform is not None:
            image = self.transform(image)
        return (image, name_label)

# Finding mean and std for Normalization

By combining the train_1.csv and test_1.csv, a full sample of the database can be loaded for optimum means and standard deviations for image normalization.

In [4]:
#dataset = ChimpNamesDataset(csv_file='train_1.csv', root_dir='chimp_faces/', transform=train_transform)

In [5]:
# loader = DataLoader(dataset,
#                          batch_size=32,
#                          num_workers=0,
#                          shuffle=False)

# mean = 0.
# std = 0.
# for images, _ in loader:
#     batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
#     images = images.view(batch_samples, images.size(1), -1)
#     mean += images.mean(2).sum(0)
#     std += images.std(2).sum(0)

# mean /= len(loader.dataset)
# std /= len(loader.dataset)

# Loading Datasets

In [6]:
batch_size = 64
train_loaders = []
test_loaders = []

ds_train1 = ChimpNamesDataset(csv_file='train_1.csv', root_dir='chimp_faces/', transform=train_transform)
ds_train2 = ChimpNamesDataset(csv_file='train_2.csv', root_dir='chimp_faces/', transform=train_transform)
ds_train3 = ChimpNamesDataset(csv_file='train_3.csv', root_dir='chimp_faces/', transform=train_transform)
ds_train4 = ChimpNamesDataset(csv_file='train_4.csv', root_dir='chimp_faces/', transform=train_transform)
ds_train5 = ChimpNamesDataset(csv_file='train_5.csv', root_dir='chimp_faces/', transform=train_transform)

ds_test1 = ChimpNamesDataset(csv_file='test_1.csv', root_dir='chimp_faces/', transform=train_transform)
ds_test2 = ChimpNamesDataset(csv_file='test_2.csv', root_dir='chimp_faces/', transform=train_transform)
ds_test3 = ChimpNamesDataset(csv_file='test_3.csv', root_dir='chimp_faces/', transform=train_transform)
ds_test4 = ChimpNamesDataset(csv_file='test_4.csv', root_dir='chimp_faces/', transform=train_transform)
ds_test5 = ChimpNamesDataset(csv_file='test_5.csv', root_dir='chimp_faces/', transform=train_transform)

train1_loader = DataLoader(dataset=ds_train1, batch_size=batch_size, shuffle=True)
train2_loader = DataLoader(dataset=ds_train2, batch_size=batch_size, shuffle=True)
train3_loader = DataLoader(dataset=ds_train3, batch_size=batch_size, shuffle=True)
train4_loader = DataLoader(dataset=ds_train4, batch_size=batch_size, shuffle=True)
train5_loader = DataLoader(dataset=ds_train5, batch_size=batch_size, shuffle=True)
train_loaders.append(train1_loader)
train_loaders.append(train2_loader)
train_loaders.append(train3_loader)
train_loaders.append(train4_loader)
train_loaders.append(train5_loader)

test1_loader = DataLoader(dataset=ds_test1, batch_size=batch_size, shuffle=True)
test2_loader = DataLoader(dataset=ds_test2, batch_size=batch_size, shuffle=True)
test3_loader = DataLoader(dataset=ds_test3, batch_size=batch_size, shuffle=True)
test4_loader = DataLoader(dataset=ds_test4, batch_size=batch_size, shuffle=True)
test5_loader = DataLoader(dataset=ds_test5, batch_size=batch_size, shuffle=True)
test_loaders.append(test1_loader)
test_loaders.append(test2_loader)
test_loaders.append(test3_loader)
test_loaders.append(test4_loader)
test_loaders.append(test5_loader)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')



# Training and Visualizing Functions

In [7]:
def check_accuracy(loader, model):
    """
    Evaluate model accuracy against a provided test dataset.
    """
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples):.2f}"
        )

    model.train()
    return float(num_correct)/float(num_samples)

# ResNet18 (ImageNet) Transfer Learning with Fine Tuning

In [8]:
def train_finetune(epochs, num_classes, train_dataloaders, test_dataloaders, model_name, subfolder_name):
    """
    Train a ResNet18 model, including the imported pretrained weights from ImageNet.
    """
    main_save_folder = 'saved_models/'
    full_save_directory = main_save_folder+subfolder_name+'/'
    accuracy_list = []
    #Loop for all five splits
    for i in range(5):
        numbered_name = ''
        print("\nSplit", str(i+1) + ":")
        numbered_name = model_name + '-' + str(i+1)+ '.pth'
        full_save_path = full_save_directory + numbered_name
        #Load pretrained ResNet18 Model
        model_ft = models.resnet18(pretrained=True)
        model_ft  = model_ft.cuda() if device else model_ft
        num_ftrs = model_ft.fc.in_features
        #Add classification layers
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        model_ft = model_ft.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adagrad(model_ft.parameters(), lr=0.001)
        step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
        #activate train mode for model
        model_ft.train()
        for epoch in range(epochs):
            losses = []
            for batch_idx, (data, targets) in enumerate(train_dataloaders[i]):
                # Get data to cuda if possible
                data = data.to(device=device)
                targets = targets.to(device=device)

                # forward
                scores = model_ft(data)
                loss = criterion(scores, targets)
                losses.append(loss.item())

                # backward
                optimizer.zero_grad()
                loss.backward()

                # gradient descent or adam step
                optimizer.step()

            print(f"Epoch {epoch} Average Loss: {sum(losses)/len(losses):.5f}")

            if epoch%5==0:
                print(f"Epoch {epoch} Training Accuracy:")
                check_accuracy(train_dataloaders[i], model_ft)
            step_lr_scheduler.step()
        print("\nEvaluation against test data:")
        accuracy = check_accuracy(test_dataloaders[i], model_ft)
        accuracy_list.append(accuracy)
        torch.save(model_ft.state_dict(), full_save_path)
    print("\n\nAverage Accuracy: ", (sum(accuracy_list)/len(accuracy_list)))

In [9]:
train_finetune(10, 86, train_loaders, test_loaders, 'resnet18-nofreeze', 'nofreeze')


Split 1:
Epoch 0 Average Loss: 2.63875
Epoch 0 Training Accuracy:
Got 2661 / 5155 with accuracy 0.52
Epoch 1 Average Loss: 1.57734
Epoch 2 Average Loss: 1.17944
Epoch 3 Average Loss: 0.92689
Epoch 4 Average Loss: 0.74597
Epoch 5 Average Loss: 0.63173
Epoch 5 Training Accuracy:
Got 4715 / 5155 with accuracy 0.91
Epoch 6 Average Loss: 0.52400
Epoch 7 Average Loss: 0.43577
Epoch 8 Average Loss: 0.43255
Epoch 9 Average Loss: 0.41960

Evaluation against test data:
Got 1101 / 1331 with accuracy 0.83

Split 2:
Epoch 0 Average Loss: 2.61400
Epoch 0 Training Accuracy:
Got 2887 / 5155 with accuracy 0.56
Epoch 1 Average Loss: 1.53815
Epoch 2 Average Loss: 1.14497
Epoch 3 Average Loss: 0.89706
Epoch 4 Average Loss: 0.72329
Epoch 5 Average Loss: 0.60995
Epoch 5 Training Accuracy:
Got 4688 / 5155 with accuracy 0.91
Epoch 6 Average Loss: 0.50521
Epoch 7 Average Loss: 0.42694
Epoch 8 Average Loss: 0.41397
Epoch 9 Average Loss: 0.40954

Evaluation against test data:
Got 1092 / 1331 with accuracy 0.82


# ResNet18 (ImageNet) Transfer Learning with no Fine Tuning

In [10]:
def train_freeze_params(epochs, num_classes, train_dataloaders, test_dataloaders, model_name, subfolder_name):
    """
    Train a ResNet18 model without making changes to the imported pretrained weights from ImageNet.
    Freeze all parameters from being trained apart from custom classification layers.
    """
    main_save_folder = 'saved_models/'
    full_save_directory = main_save_folder+subfolder_name+'/'
    accuracy_list = []
    #Loop for all five splits
    for i in range(5):
        numbered_name = ''
        print("\nSplit", str(i+1) + ":")
        numbered_name = model_name + '-' + str(i+1)+ '.pth'
        full_save_path = full_save_directory + numbered_name
        #Load pretrained ResNet18 Model
        model_freeze = models.resnet18(pretrained=True)
        for param in model_freeze.parameters():
            param.requires_grad = False
        model_freeze  = model_freeze.cuda() if device else model_freeze
        num_ftrs = model_freeze.fc.in_features
        #Add classification layers
        model_freeze.fc = nn.Linear(num_ftrs, num_classes)
        model_freeze = model_freeze.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adagrad(model_freeze.parameters(), lr=0.001)
        step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
        #activate train mode for model
        model_freeze.train()
        for epoch in range(epochs):
            losses = []
            for batch_idx, (data, targets) in enumerate(train_dataloaders[i]):
                # Get data to cuda if possible
                data = data.to(device=device)
                targets = targets.to(device=device)

                # forward
                scores = model_freeze(data)
                loss = criterion(scores, targets)
                losses.append(loss.item())

                # backward
                optimizer.zero_grad()
                loss.backward()

                # gradient descent or adam step
                optimizer.step()

            print(f"Epoch {epoch} Average Loss: {sum(losses)/len(losses):.5f}")

            if epoch%5==0:
                print(f"Epoch {epoch} Training Accuracy:")
                check_accuracy(train_dataloaders[i], model_freeze)
            step_lr_scheduler.step()
        print("\nEvaluation against test data:")
        accuracy = check_accuracy(test_dataloaders[i], model_freeze)
        accuracy_list.append(accuracy)
        torch.save(model_freeze.state_dict(), full_save_path)
    print("\n\nAverage Accuracy: ", (sum(accuracy_list)/len(accuracy_list)))

In [11]:
train_freeze_params(10, 86, train_loaders, test_loaders, 'resnet18-yesfreeze', 'yesfreeze')


Split 1:
Epoch 0 Average Loss: 4.17883
Epoch 0 Training Accuracy:
Got 375 / 5155 with accuracy 0.07
Epoch 1 Average Loss: 4.00423
Epoch 2 Average Loss: 3.91313
Epoch 3 Average Loss: 3.85037
Epoch 4 Average Loss: 3.80619
Epoch 5 Average Loss: 3.75023
Epoch 5 Training Accuracy:
Got 695 / 5155 with accuracy 0.13
Epoch 6 Average Loss: 3.71597
Epoch 7 Average Loss: 3.68375
Epoch 8 Average Loss: 3.68260
Epoch 9 Average Loss: 3.67541

Evaluation against test data:
Got 179 / 1331 with accuracy 0.13

Split 2:
Epoch 0 Average Loss: 4.20700
Epoch 0 Training Accuracy:
Got 375 / 5155 with accuracy 0.07
Epoch 1 Average Loss: 4.03309
Epoch 2 Average Loss: 3.95025
Epoch 3 Average Loss: 3.87191
Epoch 4 Average Loss: 3.81871
Epoch 5 Average Loss: 3.77729
Epoch 5 Training Accuracy:
Got 650 / 5155 with accuracy 0.13
Epoch 6 Average Loss: 3.73554
Epoch 7 Average Loss: 3.71590
Epoch 8 Average Loss: 3.70312
Epoch 9 Average Loss: 3.69736

Evaluation against test data:
Got 161 / 1331 with accuracy 0.12

Split

# Loading and Evaluating
Please disable training code by commenting it out and run all the way from the start.

In [12]:
# def load_model(load_path):
#     loaded_model = models.resnet18(pretrained=True)
#     loaded_model  = loaded_model.cuda() if device else loaded_model
#     num_ftrs = loaded_model.fc.in_features
#     #Add classification layers
#     loaded_model.fc = nn.Linear(num_ftrs, 86)
#     loaded_model = loaded_model.to(device)
#     loaded_model.load_state_dict(torch.load(load_path))
#     loaded_model.eval()
#     for parameter in loaded_model.parameters():
#         parameter.requires_grad = False

#     loaded_model.eval()
#     return loaded_model

In [13]:
# accuracy_list = []
# for i in range(5):
#     #reset model_name for next loop
#     model_name = ''
#     directory_name = 'saved_models/' + 'nofreeze' + '/'
#     model_name = 'resnet18-nofreeze' + '-' + str(i+1) +'.pth'
#     load_path = directory_name + model_name
#     model = load_model(load_path)
#     accuracy = check_accuracy(test_loaders[i], model)
#     accuracy_list.append(accuracy)

# print("\n\nAverage Accuracy: ", (sum(accuracy_list)/len(accuracy_list)))